In [1]:
import numpy as np
import os
import pandas as pd
from tqdm.notebook import tqdm
import copy

In [2]:
class Node:
    def __init__(self, value):
        self.value = value
        self.neighbors = set()
        self.parent = None
        self.level = None
        self.visited = False
    
    #Retorna o índice do vértice quando for chamado
    def __repr__(self): 
        return str(self.value)
    
    #Adiciona um vértice adjacente
    def add_neighbor(self, w):
        self.neighbors.add(w)
    
    #Retorna o índice do vértice
    def get_value(self):
        return self.value
    
    #Retorna os vértices adjacentes
    def get_neighbors(self):
        return sorted(self.neighbors, key=lambda x: x.value)

In [3]:
class Graph:
    
    def __init__(self, n):
        self.n_nodes = n
        self.matrix = np.zeros((n, n), dtype="object")
        self.nodes = np.empty(n, dtype="object")
        
        pbar = tqdm(range(n), desc="Adding edges...")
        
        for index in tqdm(range(n)):
            self.add_node(index + 1)
       

    def add_node(self, value):
        index = value - 1
        self.nodes[index] = Node(value)
    
    def add_edge(self, v_value, w_value):
        v_index = v_value - 1
        w_index = w_value - 1
        
        if v_value == w_value:
            self.matrix[v_index, v_index] = 2
            self.nodes[v_index].add_neighbor(self.nodes[v_index])
            
        else:
            self.matrix[v_index, w_index] = 1
            self.matrix[w_index, v_index] = 1
            self.nodes[v_index].add_neighbor(self.nodes[w_index])
            self.nodes[w_index].add_neighbor(self.nodes[v_index])
    
    def get_node(self, value):
        index = value - 1
        return self.nodes[index]
    
    def get_nodes(self):
        return self.nodes
    
    def get_node_edges(self):
        return {node.value:node.get_neighbors() for node in self.nodes}
    
    def get_matrix(self):
        return self.matrix
    
    def get_matrix_beautiful(self):
        return pd.DataFrame(self.matrix, columns=np.arange(1, self.n_nodes+1), index=np.arange(1, self.n_nodes+1))
    
    def get_lists(self):
        return [node.get_neighbors() for node in self.nodes]

In [4]:
def open_graph_txt(filename, extra=False):
    with open(filename, "r") as f:
        lines = f.read().split("\n")
        n_nodes = int(lines[0])
        edges = [tuple(map(lambda i: int(i), line.split(" "))) for line in lines[1:-1]]
    
    graph = Graph(n_nodes)
    for v, w in edges:
        graph.add_edge(v, w)
    
    if extra:
        return graph, n_nodes, edges
    
    return graph

In [5]:
folder = "inputs"
filename = "grafo_teste.txt"

path = os.path.join(folder, filename)

In [6]:
graph = open_graph_txt(path)

Adding edges...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
graph.get_node_edges()

{1: [2, 5], 2: [1, 5], 3: [5], 4: [5], 5: [1, 2, 3, 4]}

In [8]:
graph.get_lists()

[[2, 5], [1, 5], [5], [5], [1, 2, 3, 4]]

In [9]:
graph.get_matrix()

array([[0, 1, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [1, 1, 1, 1, 0]], dtype=object)

In [10]:
graph.get_matrix_beautiful()

,1,2,3,4,5
1,0,1,0,0,1
2,1,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1
5,1,1,1,1,0


In [11]:
def graph_statistics(graph):
    print("Número de vértices:", graph.n_nodes)
    print("Número de arestas:", graph.get_matrix().sum()/2)
    print("Grau mínimo:", graph.get_matrix().sum(axis=0).min())
    print("Grau máximo:", graph.get_matrix().sum(axis=0).max())
    print("Grau médio:", graph.get_matrix().sum(axis=0).mean())
    print("Mediana do Grau:", np.median(graph.get_matrix().sum(axis=0)))

In [12]:
graph_statistics(graph)

Número de vértices: 5
Número de arestas: 5.0
Grau mínimo: 1
Grau máximo: 4
Grau médio: 2.0
Mediana do Grau: 2.0


In [13]:
class Stack:
    
    def __init__(self, size):
        self.size = size
        self.top = -1
        
        self.values = np.empty(self.size, dtype="object")
    
    def is_full(self):
        return (self.top == self.size - 1)
    
    def is_empty(self):
        return (self.top == -1)
    
    def stack_up(self, node):
        if self.is_full():
            raise OverflowError("Stack is full.")
        else:
            self.top += 1
            self.values[self.top] = node
    
    def unstack(self):
        if self.is_empty():
            raise ValueError("Stack is empty.")
        else:
            temp = self.values[self.top]
            self.top -= 1
            return temp
    
    def get_top(self):
        if self.top != -1:
            return self.values[self.top]
        else:
            return -1
        

In [14]:
class DFS:
    def __init__(self, graph, root):
        self.graph = copy.deepcopy(graph)
        self.root = self.graph.nodes[root-1]
        self.root.level = 0
        self.root.visited = True
        self.stack = Stack(size=graph.n_nodes)
        self.stack.stack_up(self.root)
    
    def search(self):
        top = self.stack.get_top()
        for neighbor in top.get_neighbors():
            if neighbor.value == top.value:
                continue
            
            if not neighbor.visited:
                neighbor.parent = top
                neighbor.level = top.level + 1
                neighbor.visited = True
                self.stack.stack_up(neighbor)
                self.search()
        self.stack.unstack()

In [15]:
dfs = DFS(graph, 1)

In [16]:
dfs.search()

In [17]:
for node in  dfs.graph.nodes[:5]:
    print("Node:", node.value)
    print("Parent:", node.parent)
    print("Level:", node.level)
    print("Visited:", node.visited)
    print("----")


Node: 1
Parent: None
Level: 0
Visited: True
----
Node: 2
Parent: 1
Level: 1
Visited: True
----
Node: 3
Parent: 5
Level: 3
Visited: True
----
Node: 4
Parent: 5
Level: 3
Visited: True
----
Node: 5
Parent: 2
Level: 2
Visited: True
----


In [20]:
np.empty(5)[1:]

array([8.01097889e-307, 1.24612081e-306, 1.60220528e-306, 1.69119330e-306])

In [62]:
class Queue:
    def __init__(self, size):
        self.size = size
        self.top = -1
        
        self.queue = np.empty(self.size, dtype="object")
    
    def is_full(self):
        return (self.top == self.size - 1)
    
    def is_empty(self):
        return (self.top == -1)
    
    def add(self, node):
        if self.is_full and not self.is_empty:
            raise OverflowError("Queue is full.")
        self.top += 1
        self.queue[self.top] = node

    def next(self):
        if self.is_empty():
            raise ValueError("Queue is empty.")
        first = self.queue[0]
        self.queue = self.queue[1:]
        self.top -= 1
        np.append(self.queue, None)
        return first


    def get(self):
        return self.queue

In [63]:
class BFS:
    def __init__(self, graph, root):
        self.graph = copy.deepcopy(graph)
        self.root = self.graph.nodes[root-1]
        self.root.level = 0
        self.root.visited = True
        self.queue = Queue(size=graph.n_nodes)
        self.queue.add(self.root)
    
    def search(self):
        first = self.queue.next()
        print(first)
        print("Queue:", self.queue.queue)
        
        for neighbor in first.get_neighbors():
            print("neighbor:", neighbor)
            
            if first == neighbor:
                continue
            if not neighbor.visited:
                neighbor.visited = True
                neighbor.parent = first
                neighbor.level = first.level + 1
                self.queue.add(neighbor)
            print("Queue:", self.queue.queue)
        if not self.queue.is_empty():
            self.search()

In [64]:
bfs = BFS(graph, 1)

In [65]:
bfs.search()

1
Queue: [None None None None]
neighbor: 2
Queue: [2 None None None]
neighbor: 5
Queue: [2 5 None None]
2
Queue: [5 None None]
neighbor: 1
Queue: [5 None None]
neighbor: 5
Queue: [5 None None]
5
Queue: [None None]
neighbor: 1
Queue: [None None]
neighbor: 2
Queue: [None None]
neighbor: 3
Queue: [3 None]
neighbor: 4
Queue: [3 4]
3
Queue: [4]
neighbor: 5
Queue: [4]
4
Queue: []
neighbor: 5
Queue: []


In [66]:
for node in  bfs.graph.nodes[:5]:
    print("Node:", node.value)
    print("Parent:", node.parent)
    print("Level:", node.level)
    print("Visited:", node.visited)
    print("----")


Node: 1
Parent: None
Level: 0
Visited: True
----
Node: 2
Parent: 1
Level: 1
Visited: True
----
Node: 3
Parent: 5
Level: 2
Visited: True
----
Node: 4
Parent: 5
Level: 2
Visited: True
----
Node: 5
Parent: 1
Level: 1
Visited: True
----


In [30]:
from pyvis.network import Network

net = Network(notebook=True)

for node in graph.nodes:
    net.add_node(node.value, label=node.value)
    
for node in graph.nodes:
    for neighbor in node.neighbors:
        net.add_edge(node.value, neighbor.value)

net.show("graph.html")